In [57]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model
import json
import collections
import copy

assignment = pd.read_csv("assignment.csv")
distances = pd.read_csv("brick_rp_distances.csv")
workload = pd.read_csv("bricks_index_values.csv")

In [58]:
def str_to_list(to_cast):
    return json.loads(to_cast)

In [59]:
def create_init_assign_matrix(assignments):
    assignment_d = {}
    for i in range(4):
        x = assignment[assignment["SR#"] == (i+1)]
        for brick in str_to_list(x["Bricks_Assigned"][i]):
            assignment_d[brick] = i + 1
    sorted_dict = dict(sorted(assignment_d.items()))
    matrix = [[int((i+1) == sorted_dict[k])for i in range(4)] for k in sorted_dict.keys()]
    return matrix

In [60]:
def create_index_value_matrix(bricks_index_values):
    index_dict = bricks_index_values.set_index("brick").T.to_dict('list')
    sorted_dict = dict(sorted(index_dict.items()))
    return [[float(sorted_dict[index][0]), float(sorted_dict[index][0]), float(sorted_dict[index][0]), float(sorted_dict[index][0])] for index in sorted_dict.keys()]

In [61]:
def create_distances_matrix(distances):
    distances_dict = distances.set_index("brick").T.to_dict('list')
    sorted_dict = dict(sorted(distances_dict.items()))
    return [sorted_dict[index] for index in sorted_dict.keys()]

In [62]:
def print_solution(sol):
    for brick in range(len(sol)):
        print(f"{brick + 1} : {[sol[brick][sr] for sr in range(len(sol[0]))]}")

In [63]:
def compute_reassignments(init, solution):
    return 1/2*sum([init[brick][sr] - 2*init[brick][sr]*solution[brick][sr] + solution[brick][sr] for brick in range(len(init)) for sr in range(len(init[0]))])

In [64]:
def compute_total_distances(distances_m, solution):
    dist = 0
    for sr in range(len(distances_m[0])):
        for brick in range(len(distances_m)):
            dist += distances_m[brick][sr] * solution[brick][sr]
    return dist

In [65]:
def print_reassignments(init, solution):
    for brick in range(len(init)):
        first_sr = -1
        second_sr = -1
        for sr in range(len(init[0])):
            if init[brick][sr] - solution[brick][sr] == 1:
                first_sr = sr
            if init[brick][sr] - solution[brick][sr] == -1:
                second_sr = sr
        if first_sr != -1 and second_sr != -1:
            print(f"Brick {brick + 1} has changed from SR {first_sr + 1} to SR {second_sr + 1}")

In [66]:
def print_distance_by_sr(distances_m, solution):
    for sr in range(len(distances_m[0])):
        dist = 0
        for brick in range(len(distances_m)):
            dist += distances_m[brick][sr] * solution[brick][sr]
        print(f"SR {sr + 1} is travelling {dist}km")

In [67]:
def print_workload_by_sr(workload_m, solution):
    for sr in range(len(workload_m[0])):
        wl = 0
        for brick in range(len(workload_m)):
            wl += workload_m[brick][sr] * solution[brick][sr]
        print(f"SR {sr + 1} is working {wl}")

In [68]:
# Create linear solver
solver = pywraplp.Solver.CreateSolver("SCIP")
if not solver:
    print("Error during solver's creation")

In [69]:
# Values for constraints
epsilon = 0.1
min_workload = 0.8
max_workload = 1.2
init_state_matrix = create_init_assign_matrix(assignment)
index_value_matrix = create_index_value_matrix(workload)
distances_matrix = create_distances_matrix(distances)
hq_bricks = [[0,3],[1,13],[2,15],[3,21]]
num_sr = len(distances_matrix[0])
num_bricks = len(distances_matrix)

# Boolean variables
x = {}
for brick in range(num_bricks):
    for sr in range(num_sr):
        x[brick, sr] = solver.IntVar(0, 1, f"x[{brick},{sr}]")

# Add constraints
# One brick to one SR and one SR to one brick
for brick in range(num_bricks):
    solver.Add(solver.Sum(x[brick, sr] for sr in range(num_sr)) == 1)

# The cental brick for an SR cannot change
hq_vars = []
for hq in hq_bricks:
    hq_vars.append(x[hq[1],hq[0]])
solver.Add(solver.Sum(hq_vars) == num_sr)

# Balance workload (min and max)
for sr in range(num_sr):
    solver.Add(solver.Sum(index_value_matrix[brick][sr] * x[brick,sr] for brick in range(num_bricks)) >= min_workload)
    solver.Add(solver.Sum(index_value_matrix[brick][sr] * x[brick,sr] for brick in range(num_bricks)) <= max_workload)

In [70]:
solutions = {}
feasible = True
loop = 0
while feasible:
    if loop % 2:
        if loop != 0:
            solver.Add(solver.Sum((x[brick,sr] - 2*init_state_matrix[brick][sr]*x[brick,sr] + init_state_matrix[brick][sr]) * 1/2 for brick in range(num_bricks) for sr in range(num_sr)) <= solutions[loop-1][2] - epsilon)
        solver.Minimize(
            solver.Sum((x[brick,sr] - 2*init_state_matrix[brick][sr]*x[brick,sr] + init_state_matrix[brick][sr]) * 1/2  for brick in range(num_bricks) for sr in range(num_sr))
        )
    else:
        if loop != 0:
            solver.Add(solver.Sum(distances_matrix[brick][sr] * x[brick,sr] for sr in range(num_sr) for brick in range(num_bricks)) <= solutions[loop-1][1] - epsilon)
        solver.Minimize(
            solver.Sum(distances_matrix[brick][sr] * x[brick,sr] for sr in range(num_sr) for brick in range(num_bricks))
        )

    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print(f"################## LOOP {loop + 1} ###################") 
        solution = [[x[brick,sr].solution_value() for sr in range(num_sr)] for brick in range(num_bricks)]
        print_reassignments(init_state_matrix, solution)
        print()
        print("======DISTANCES======")
        print_distance_by_sr(distances_matrix, solution)
        print()
        print("======WORKLOAD======")
        print_workload_by_sr(index_value_matrix, solution)
        print()
        print(f"Number of reassignments : {compute_reassignments(init_state_matrix, solution)}")
        print(f"Total distance : {compute_total_distances(distances_matrix, solution)}")
        print()
        solutions[loop] = [solution, compute_total_distances(distances_matrix, solution), compute_reassignments(init_state_matrix, solution)]
    else:
        if status == 2:
            feasible = False
    loop += 1

################## LOOP 1 ###################
Brick 9 has changed from SR 3 to SR 1
Brick 10 has changed from SR 2 to SR 3
Brick 12 has changed from SR 2 to SR 1
Brick 15 has changed from SR 1 to SR 3
Brick 18 has changed from SR 3 to SR 2
Brick 19 has changed from SR 4 to SR 1
Brick 20 has changed from SR 4 to SR 1

======DISTANCES======
SR 1 is travelling 64.37km
SR 2 is travelling 7.559999999999999km
SR 3 is travelling 6.5600000000000005km
SR 4 is travelling 76.13km

======WORKLOAD======
SR 1 is working 1.0375999999999999
SR 2 is working 1.0447
SR 3 is working 1.1149
SR 4 is working 0.8028000000000001

Number of reassignments : 7.0
Total distance : 154.62

################## LOOP 2 ###################
Brick 10 has changed from SR 2 to SR 3

======DISTANCES======
SR 1 is travelling 19.3km
SR 2 is travelling 28.800000000000004km
SR 3 is travelling 14.36km
SR 4 is travelling 124.74000000000001km

======WORKLOAD======
SR 1 is working 0.9507
SR 2 is working 1.0848
SR 3 is working 0.9577


In [77]:
# Create linear solver
solver = pywraplp.Solver.CreateSolver("SCIP")
if not solver:
    print("Error during solver's creation")

In [78]:
# Values for constraints
epsilon = 0.1
min_workload = 0.9
max_workload = 1.1
init_state_matrix = create_init_assign_matrix(assignment)
index_value_matrix = create_index_value_matrix(workload)
distances_matrix = create_distances_matrix(distances)
hq_bricks = [[0,3],[1,13],[2,15],[3,21]]
num_sr = len(distances_matrix[0])
num_bricks = len(distances_matrix)

# Boolean variables
x = {}
for brick in range(num_bricks):
    for sr in range(num_sr):
        x[brick, sr] = solver.IntVar(0, 1, f"x[{brick},{sr}]")

# Add constraints
# One brick to one SR and one SR to one brick
for brick in range(num_bricks):
    solver.Add(solver.Sum(x[brick, sr] for sr in range(num_sr)) == 1)

# The cental brick for an SR cannot change
hq_vars = []
for hq in hq_bricks:
    hq_vars.append(x[hq[1],hq[0]])
solver.Add(solver.Sum(hq_vars) == num_sr)

# Balance workload (min and max)
for sr in range(num_sr):
    solver.Add(solver.Sum(index_value_matrix[brick][sr] * x[brick,sr] for brick in range(num_bricks)) >= min_workload)
    solver.Add(solver.Sum(index_value_matrix[brick][sr] * x[brick,sr] for brick in range(num_bricks)) <= max_workload)

In [79]:
solutions = {}
feasible = True
loop = 0
while feasible:
    if loop % 2:
        if loop != 0:
            solver.Add(solver.Sum((x[brick,sr] - 2*init_state_matrix[brick][sr]*x[brick,sr] + init_state_matrix[brick][sr]) * 1/2 for brick in range(num_bricks) for sr in range(num_sr)) <= solutions[loop-1][2] - epsilon)
        solver.Minimize(
            solver.Sum((x[brick,sr] - 2*init_state_matrix[brick][sr]*x[brick,sr] + init_state_matrix[brick][sr]) * 1/2  for brick in range(num_bricks) for sr in range(num_sr))
        )
    else:
        if loop != 0:
            solver.Add(solver.Sum(distances_matrix[brick][sr] * x[brick,sr] for sr in range(num_sr) for brick in range(num_bricks)) <= solutions[loop-1][1] - epsilon)
        solver.Minimize(
            solver.Sum(distances_matrix[brick][sr] * x[brick,sr] for sr in range(num_sr) for brick in range(num_bricks))
        )

    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print(f"################## LOOP {loop + 1} ###################") 
        solution = [[x[brick,sr].solution_value() for sr in range(num_sr)] for brick in range(num_bricks)]
        print_reassignments(init_state_matrix, solution)
        print()
        print("======DISTANCES======")
        print_distance_by_sr(distances_matrix, solution)
        print()
        print("======WORKLOAD======")
        print_workload_by_sr(index_value_matrix, solution)
        print()
        print(f"Number of reassignments : {compute_reassignments(init_state_matrix, solution)}")
        print(f"Total distance : {compute_total_distances(distances_matrix, solution)}")
        print()
        solutions[loop] = [solution, compute_total_distances(distances_matrix, solution), compute_reassignments(init_state_matrix, solution)]
    else:
        if status == 2:
            feasible = False
    loop += 1

################## LOOP 1 ###################
Brick 9 has changed from SR 3 to SR 1
Brick 10 has changed from SR 2 to SR 3
Brick 12 has changed from SR 2 to SR 1
Brick 15 has changed from SR 1 to SR 3
Brick 17 has changed from SR 3 to SR 2
Brick 19 has changed from SR 4 to SR 1

======DISTANCES======
SR 1 is travelling 46.88km
SR 2 is travelling 9.19km
SR 3 is travelling 7.0200000000000005km
SR 4 is travelling 99.34km

======WORKLOAD======
SR 1 is working 0.9379
SR 2 is working 1.073
SR 3 is working 1.0866
SR 4 is working 0.9025

Number of reassignments : 6.0
Total distance : 162.43

################## LOOP 2 ###################
Brick 10 has changed from SR 2 to SR 3

======DISTANCES======
SR 1 is travelling 19.3km
SR 2 is travelling 28.800000000000004km
SR 3 is travelling 14.36km
SR 4 is travelling 124.74000000000001km

======WORKLOAD======
SR 1 is working 0.9507
SR 2 is working 1.0848
SR 3 is working 0.9577
SR 4 is working 1.0068000000000001

Number of reassignments : 1.0
Total dista